In [103]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

%matplotlib qt

In [104]:
cap = cv.VideoCapture('video/traffic.mp4')

In [105]:
x, y, w, h = 395, 0, 30, 30
track_window = (x, y, w, h)
ret, frame = cap.read()
frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
plt.figure()
plt.imshow(frame_rgb)
plt.show()
roi = frame[y:y+h, x:x+w]

In [106]:
hsv_roi = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask = cv.inRange(roi, np.array((135., 122., 252.)), np.array((255., 255., 255.)))
roi_hist = cv.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)
term_crit = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1)

In [107]:
cv.namedWindow('traffic', cv.WINDOW_NORMAL)
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)
        ret, track_window = cv.meanShift(dst, track_window, term_crit)
        x, y, w, h = track_window
        final_image = cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv.imshow('traffic', final_image)
    else:
        cap.set(cv.CAP_PROP_POS_FRAMES, 0)
        continue

    if cv.waitKey(10) & 0xFF == ord('q'):
        break

In [108]:
cap.release()
cv.destroyAllWindows()